In [36]:
import numpy as np
import pandas as pd
import seaborn as sns
import mesa
import random

In [37]:
class MoneyAgent(mesa.Agent):
    """An agent with fixed initial wealth."""

    def __init__(self, model):
        # Pass the parameters to the parent class.
        super().__init__(model)

        # Create the agent's attribute and set the initial values.
        self.wealth = 1

    def say_hi(self):
        # The agent's step will go here.
        # For demonstration purposes we will print the agent's unique_id
        print(f"Hi, I am an agent, you can call me {self.unique_id!s}.")

class Household(mesa.Agent):

    def __init__(self, model):
        super().__init__(model)

        self.money = 50
        self.utility = 0
        self.type = "H"
        self.goods = 0

    def decide_actions(self):

        self.consumption = random.random()
        self.laborh = random.random() * 10

    def calculate_utility(self):
        self.utility = self.goods * 100 - self.laborh * 10 + self.money * 20
        self.goods = 0
        print("I am household ",self.unique_id," My utility: ",self.utility)

class Firm(mesa.Agent):

    def __init__(self, model):
        super().__init__(model)

        self.money = 50
        self.utility = 0
        self.inventory = 0
        self.revenue = 0
        self.type = "F"

    def decide_actions(self):
        print("i am firm")
        self.salary = random.random() * 10
        self.price = random.random() * 10

    def calculate_utility(self):
        self.utility = self.revenue * 100 - self.salary * 10 - self.inventory * 20
        self.revenue = 0
        print("I am firm ",self.unique_id," My utility: ",self.utility)

class MoneyModel(mesa.Model):
    """A model with some number of agents."""

    def __init__(self,num_hoseholds,num_firms,seed=None):
        super().__init__(seed=seed)
        self.num_households = num_hoseholds
        self.num_firms = num_firms

        # Create n agents
        Household.create_agents(model=self,n = num_hoseholds)
        Firm.create_agents(model=self,n = num_firms)

    def step(self):
        self.agents.do("decide_actions")

        # consequences


        # production
        total_work_hours = 0
        total_consumption = 0
        for agent in self.agents.select(lambda a : a.type == "H"):
            total_work_hours += agent.laborh
            total_consumption += agent.consumption * agent.money
            
        total_salary = 0
        higest_price = 0
        for firm in self.agents.select(lambda a : a.type == "F"):
            total_salary += firm.salary
            higest_price = max(higest_price,firm.price)

        total_money_earned = 0
        production_constant = 1
        total_price_inverse = 0
        for firm in self.agents.select(lambda a : a.type == "F"):
            total_price_inverse += higest_price - firm.price
            total_money_earned += total_work_hours * (firm.salary / total_salary) * firm.salary

            firm.inventory = firm.inventory + total_work_hours * (firm.salary / total_salary) * production_constant
            firm.money -= total_work_hours * (firm.salary / total_salary) * firm.salary

        

        # consumption
        total_consumption_left = total_consumption
        total_goods_bought = 0
        for firm in self.agents.select(lambda a : a.type == "F"):

            firm.revenue = max( total_consumption * ((higest_price - firm.price)/total_price_inverse) * firm.price, firm.price * firm.inventory)
            firm.money += firm.revenue
            total_consumption_left -= firm.revenue
            total_goods_bought += firm.revenue / firm.price
            firm.inventory = firm.inventory - firm.revenue / firm.price

        for household in self.agents.select(lambda a : a.type == "H"):
            money_spent = household.money*household.consumption

            household.goods = (money_spent/total_consumption) * total_goods_bought
            household.money = household.money * (1 - household.consumption)
            household.money += (money_spent/total_consumption) * total_consumption_left


        # households get paid after consumption
        for household in self.agents.select(lambda a : a.type == "H"):
            household.money = household.money + total_money_earned * (household.laborh / total_work_hours)

        self.agents.do("calculate_utility")



In [38]:
starter_model = MoneyModel(2,2)
starter_model.step()

i am firm
i am firm
I am household  1  My utility:  1313.77380227322
I am household  2  My utility:  1356.073314170362
I am firm  3  My utility:  28282.320032104846
I am firm  4  My utility:  7550.284268824157


In [55]:
starter_model.step()

i am firm
i am firm
I am household  1  My utility:  -135317.92147035262
I am household  2  My utility:  80678.18540827851
I am firm  3  My utility:  1652.52276415605
I am firm  4  My utility:  77626.73021803673
